In [1]:
from openai import OpenAI

In [2]:
# Connection parameters (only has to be executed once during a session)

# Note: `localhost` is only valid as long as you are logged in on madagaskarweihe. 
# Otherwise, you will need to perform port forwarding on your machine using `ssh -fNL 8006:127.0.0.1:8006 ashousaa@madagaskarweihe`.
# This might require an ssh key to be uploaded to madagaskarweihe first.
VLLM_API_ENDPOINT = 'http://localhost:8006/v1' 
VLLM_KEY = 's7Vtzeyq3kfhVkPhlWdL95pPRBq36KDP1d5bBj54BqQ'
MODEL = "default"

In [3]:
# Connection to LLM server (only has to be executed once during a session)
client = OpenAI(api_key=VLLM_KEY,
                base_url=VLLM_API_ENDPOINT)

In [8]:
import csv

# Read the CSV with questions, answers, and chunks
with open('modified_final_test_set.csv', 'r', encoding='utf-8') as infile, open('poor_language_answers.csv', 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ['Question', 'Bad_Answer', 'Chunk']  # Columns to output
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in reader:
        question = row['Question']
        answer = row['Answer']
        chunk = row['Chunk']
        
        # Generate bad version of the answer using GPT-4
        messages = [
            {
                "role": "system",
                "content": (
                    "You are an assistant that gives plausible but poor-quality answers. "
                    "Rewrite the given answer so that it seems like it's trying to help but is actually:\n"
                    "- Incoherent: The ideas should not flow logically or clearly.\n"
                    "- Unclear: Use vague, ambiguous, or overly complex language.\n"
                    "- Verbose: Include redundant or off-topic information.\n"
                    "DO NOT include any introductory phrases, disclaimers, or meta comments like 'Here is a bad answer' or 'As requested'. "
                    "Only return the poorly written answer directly, with no additional commentary."
                )
            },
            {
                "role": "user",
                "content": f"Rewrite this answer badly:\n\n{answer}"
            }
        ]
        
        response = client.chat.completions.create(
            model=MODEL,  
            messages=messages,
            temperature=0.7, 
            max_tokens=200
        )

        # Extract the "bad" answer from the response
        bad_answer = response.choices[0].message.content.strip()

        # Write the new row with the question, bad answer, and chunk to the new CSV file
        writer.writerow({
            'Question': question,
            'Bad_Answer': bad_answer,
            'Chunk': chunk
        })
